In [1]:
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import numpy as np
import csv

### Se carga el archivo del set de entrenamiento

In [2]:
with open('../input/train.csv', 'r') as trainFile :
    trainReader = csv.reader(trainFile, delimiter=',', quotechar='"')
    trainFile.next()
    trainData = []
    for row in trainReader :
        trainTarget = int(row[0])
        rowData = np.zeros(28*28)
        for i in xrange(28*28) :
            rowData[i] = row[i+1]
                
        trainData.append((trainTarget,rowData))
                

In [3]:
trainDataRDD = sc.parallelize(trainData,64)

## A continuacion se filtran los datos para que sean procesados por clase

In [4]:
trainDataByClasses = []
for i in xrange(10):
    trainDataByClassesRDD = trainDataRDD.filter(lambda tupla : tupla[0] == i)
    trainDataByClasses.append(trainDataByClassesRDD)


### Se calcula el promedio de densidad correspondiente a cada clase

In [5]:
def weightOfImage(row):
    summarize = 0
    i,j = 0,0
    for value in row :
        if value != 0 :
            j+=1
        i+=1
    return float(j)/float(i)


#### Se crea una tupla del tipo (value,array,weight)

In [6]:
weightImages = []
for i in xrange(10):
    rddIndex = trainDataByClasses[i]
    rddAlgo = rddIndex.map(lambda tupla: (tupla[0],tupla[1],weightOfImage(tupla[1])))
    weightImages.append(rddAlgo)
    


### Se reduce un %70 la cantidad de datos de nuestro set de entrenamiento, en base a la densidad de la imagen.

In [7]:
reducedDataSet = []
for i in xrange(10) :
    rowRDD = weightImages[i]
    lenRow = rowRDD.count()
    reducedDataSet.append(rowRDD.takeOrdered(int(lenRow*0.80),lambda threepla: -threepla[2]))

### Se crea la nueva lista que contiene el set de datos modificado para usar KNN

In [8]:
listReducedDataSet = []
for i in xrange(10):
    for j in xrange(len(reducedDataSet[i])):
        listReducedDataSet.append((reducedDataSet[i][j][0],reducedDataSet[i][j][1]))

### Se crean los numpy array para poder trabajar con KNN

In [9]:
trainData = []
trainTarget = np.zeros(len(listReducedDataSet))
j = 0
for tupla in listReducedDataSet :
    array = np.zeros(28*28)
    trainTarget[j] = tupla[0]
    for i in xrange(28*28):
        array[i] = tupla[1][i]
    trainData.append(array)
    j+=1
    

In [10]:
knnDigitRecorgnize = KNeighborsClassifier(n_neighbors=3,p=4,n_jobs=2)
knnDigitRecorgnize.fit(trainData,trainTarget)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=2, n_neighbors=3, p=4,
           weights='uniform')

In [11]:
with open('../input/test.csv', 'r') as testFile :
    testReader = csv.reader(testFile, delimiter=',', quotechar='"')
    testFile.next()
    testData = []
    for row in testReader :
        rowData = np.zeros(28*28)
        for i in xrange(28*28) :
            rowData[i] = row[i]
                
        testData.append(rowData)

In [12]:
predictionFile = open('../input/knnReducedDistance3MintkP3.csv','w')

In [13]:
for row in testData :
    predictionFile.write(str(int(knnDigitRecorgnize.predict([row])[0])) + '\n')

KeyboardInterrupt: 

In [ ]:
predictionFile.close()